In [1]:
import os
import time
import warnings
warnings.filterwarnings("ignore")

import duckdb
import requests
import PyBioMed
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt

from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import OneHotEncoder

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import BitVectToText
from rdkit.Chem import Descriptors, rdFingerprintGenerator

from PyBioMed.Pyprotein import PyProtein
from PyBioMed.PyGetMol import GetProtein

In [2]:
data_dir = "/home/pervinco/Datasets/leash-bio"
save_dir = f"{data_dir}/split_sets"

train_csv = f"{data_dir}/train.csv"
test_csv = f"{data_dir}/test.csv"

train_parquet = f"{data_dir}/train.parquet"
test_parquet = f'{data_dir}/test.parquet'

target_proteins = ['sEH', 'HSA', 'BRD4']

# 1.데이터셋 분석

In [3]:
con = duckdb.connect()

## binds=0인 데이터만 load
count_binds_0 = con.query(f"""SELECT COUNT(*) 
                              FROM parquet_scan('{train_parquet}') 
                              WHERE binds = 0""").fetchone()[0]
print(f"Total binds=0 : {count_binds_0}")

## binds=1인 데이터만 load
count_binds_1 = con.query(f"""SELECT COUNT(*) 
                              FROM parquet_scan('{train_parquet}') 
                              WHERE binds = 1""").fetchone()[0]
print(f"Total binds=1 : {count_binds_1}")

## 전체 데이터 수
total_count = count_binds_0 + count_binds_1
print(f"Total data : {total_count}")

con.close()

Total binds=0 : 293656924
Total binds=1 : 1589906
Total data : 295246830


In [4]:
## 컬럼별 고유한 값, 갯수 파악

columns = [
    'buildingblock1_smiles', 
    'buildingblock2_smiles', 
    'buildingblock3_smiles', 
    'molecule_smiles', 
    'protein_name'
]

con = duckdb.connect()
for column in columns:
    query = f"SELECT {column}, COUNT(*) as count FROM parquet_scan('{train_parquet}') GROUP BY {column}"
    df = con.query(query).df()

    df.to_csv(f"{data_dir}/info/{column}_info.csv", index=False)

con.close()

In [5]:
## 컬럼별 중복 데이터 확인

con = duckdb.connect()
bb1_query = f"SELECT DISTINCT buildingblock1_smiles FROM parquet_scan('{train_parquet}')"
bb2_query = f"SELECT DISTINCT buildingblock2_smiles FROM parquet_scan('{train_parquet}')"
bb3_query = f"SELECT DISTINCT buildingblock3_smiles FROM parquet_scan('{train_parquet}')"

bb1_set = set(con.query(bb1_query).df()['buildingblock1_smiles'])
bb2_set = set(con.query(bb2_query).df()['buildingblock2_smiles'])
bb3_set = set(con.query(bb3_query).df()['buildingblock3_smiles'])

bb1_bb2_intersection = bb1_set.intersection(bb2_set)
bb1_bb3_intersection = bb1_set.intersection(bb3_set)
bb2_bb3_intersection = bb2_set.intersection(bb3_set)

print(f"Building block 1 & 2 중복 : {'있음' if bb1_bb2_intersection else '없음'}")
print(f"Building block 1 & 3 중복 : {'있음' if bb1_bb3_intersection else '없음'}")
print(f"Building block 2 & 3 중복 : {'있음' if bb2_bb3_intersection else '없음'}")

print(f"Building block 1과 2 사이의 중복된 값: {bb1_bb2_intersection}")
print(f"Building block 1과 3 사이의 중복된 값: {bb1_bb3_intersection}")
print(f"Building block 2와 3 사이의 중복된 값: {bb2_bb3_intersection}")

con.close()

Building block 1 & 2 중복 : 없음
Building block 1 & 3 중복 : 없음
Building block 2 & 3 중복 : 있음
Building block 1과 2 사이의 중복된 값: set()
Building block 1과 3 사이의 중복된 값: set()
Building block 2와 3 사이의 중복된 값: {'Cc1cc(N)nn1C', 'Nc1ccc(F)cc1F', 'Nc1ccc(N2CCOCC2)c(F)c1', 'N#Cc1ncc(N)cc1C(F)(F)F', 'Cc1cccc(OCCCN)c1C.Cl', 'Nc1nc(Cl)ncc1Cl', 'CS(=O)(=O)Nc1cccc(N)c1', 'NCc1cccnc1N1CCC(C(N)=O)CC1', 'CN(C)c1nc(Cl)c(CN)s1.Cl.Cl', 'Nc1ccc2ncccc2c1', 'N#Cc1cc(N)ccc1F', 'Cc1cc(N)ncn1', 'Cc1cc(N)cc(Cl)c1', 'NCC1CN2CCN1CC2', 'Cl.Cl.NCc1nc2c(F)c(F)ccc2[nH]1', 'COC(=O)c1cc(N)ccc1C', 'Cl.NCC1CNC(=O)C1', 'Nc1ccc2c(c1)OCCCO2', 'COc1cncc(N)c1', 'CC(C)NC(=O)NCCN.Cl.Cl', 'CC(=O)c1ccc(N)c(F)c1', 'Nc1ccc(-c2cnco2)cc1', 'C=CCOC(C)CN', 'Nc1ccc2[nH]ncc2c1', 'Cl.NCC1(F)CCOC1', 'Cc1ccc(Cl)c(N)c1', 'Nc1cccc(N2CCOCC2)c1', 'Cl.Cl.NCCn1cccnc1=O', 'COC(=O)c1ccc(N)cc1O', 'Cl.NCc1nnc(-c2ccncc2)[nH]1', 'C#Cc1cccc(N)c1', 'Cl.NCC1Cc2ccccc2C1', 'CC(C)(C)c1ccc(O)c(N)c1', 'Cc1cc(N)ncc1Cl', 'Cc1nc(Cl)ccc1N', 'Cl.Cl.NCC1(c2ccncc2)CC1', 'Nc1nnc(S)

In [6]:
limit = 2000

con = duckdb.connect()
data = con.query(f"""(SELECT *
                      FROM parquet_scan('{train_parquet}')
                      WHERE binds = 0
                      LIMIT {limit})
                      UNION ALL
                      (SELECT *
                      FROM parquet_scan('{train_parquet}')
                      WHERE binds = 1
                      LIMIT {limit})""").df()

con.close()
data.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,BRD4,0
1,1,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,HSA,0
2,2,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,sEH,0
3,3,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,BRD4,0
4,4,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,HSA,0


# 2.RDKit을 활용한 분석

In [7]:
fpg = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

## molecule을 Morgan FingerPrint로 변환.
def compute_fingerprint(mol):
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    return BitVectToText(fp)  # Convert to BitString for storage

In [8]:
## molecule로부터 descriptor 계산.
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}
    descriptors = Descriptors.CalcMolDescriptors(mol)
    return descriptors

In [9]:
def process_row(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {'fingerprint': None, 'descriptors': {}}
    fingerprint = compute_fingerprint(mol)
    descriptors = calculate_descriptors(smiles)
    return {'fingerprint': fingerprint, 'descriptors': descriptors}

In [10]:
"""
molecule_smiles는 building block들로 조합된 약물 분자.
모델 학습을 위해서는 인코딩을 적용해 컴퓨터가 이해할 수 있는 형태로 변환해야함.
"""

OFFSET = 0
CHUNK_SIZE = 1000
train_parquet = f'{data_dir}/train.parquet'
con = duckdb.connect()

output_dir = f"{data_dir}/preprocessed"
output_file = f"{output_dir}/train.parquet"
os.makedirs(output_dir, exist_ok=True)

num_workers = cpu_count()
pool = Pool(num_workers)

first_chunk = True
while OFFSET < 3000:
    chunk = con.execute(f"""
    SELECT *
    FROM parquet_scan('{train_parquet}')
    LIMIT {CHUNK_SIZE} OFFSET {OFFSET}
    """).fetch_df()

    if chunk.empty:
        break

    smiles_list = chunk['molecule_smiles'].tolist()
    
    ## 병렬로 데이터 처리
    results = pool.map(process_row, smiles_list)

    ## 결과를 데이터프레임으로 변환
    fingerprints = [result['fingerprint'] for result in results]
    descriptors_list = [result['descriptors'] for result in results]
    
    chunk['fingerprints'] = fingerprints
    descriptor_df = pd.DataFrame(descriptors_list)
    excluded_descriptors = descriptor_df.columns[descriptor_df.isna().any()].tolist()
    descriptor_df.drop(columns=excluded_descriptors, inplace=True)
    used_descriptor = descriptor_df.columns.tolist()

    if first_chunk:
        print(f"제외된 descriptors: {excluded_descriptors}")
        print(f"사용된 descriptors: {used_descriptor}")

    chunk = pd.concat([chunk, descriptor_df], axis=1)
    table = pa.Table.from_pandas(chunk)

    if first_chunk:
        writer = pq.ParquetWriter(output_file, table.schema)
        first_chunk = False

    writer.write_table(table)
    print(f"Processed offset: {OFFSET} saved to {output_file}")
    OFFSET += CHUNK_SIZE

pool.close()
pool.join()

writer.close()
con.close()

제외된 descriptors: ['BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW']
사용된 descriptors: ['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA

In [11]:
df = pd.read_parquet(output_file, engine='pyarrow')
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,fingerprints,MaxAbsEStateIndex,MaxEStateIndex,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,BRD4,0,0100000000000000000000000000000000000000010000...,12.022584,12.022584,...,0,0,0,2,0,0,0,0,0,0
1,1,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,HSA,0,0100000000000000000000000000000000000000010000...,12.022584,12.022584,...,0,0,0,2,0,0,0,0,0,0
2,2,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.Br.NCC1CCCN1c1cccnn1,C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...,sEH,0,0100000000000000000000000000000000000000010000...,12.022584,12.022584,...,0,0,0,2,0,0,0,0,0,0
3,3,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,BRD4,0,0100000000000000000000000000000000000000010000...,11.915654,11.915654,...,0,0,0,2,0,0,0,0,0,0
4,4,C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21,C#CCOc1ccc(CN)cc1.Cl,Br.NCc1cccc(Br)n1,C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...,HSA,0,0100000000000000000000000000000000000000010000...,11.915654,11.915654,...,0,0,0,2,0,0,0,0,0,0


In [ ]:
print(df.columns)
print(df.shape)

# 3.Target Protein Descriptor

In [ ]:
uniprot_dicts = {"sEH": "P34913", "BRD4": "O60885", "HSA": "P02768"}
output_dir = f"{data_dir}/protein_desc"
os.makedirs(output_dir, exist_ok=True)

def get_protein_sequence(uniprot_id):
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    response = requests.get(url)
    if response.status_code == 200:
        response_text = response.text
        lines = response_text.splitlines()
        seq = "".join(lines[1:])
        return seq
    else:
        return None

protein_seq_dicts = {}
for protein_name, uniprot_id in uniprot_dicts.items():
    protein_sequence = get_protein_sequence(uniprot_id)
    if protein_sequence:
        protein_seq_dicts[protein_name] = protein_sequence
    else:
        print(f"Failed to retrieve sequence for {protein_name} ({uniprot_id})")

ctd_features = []
for protein_name, sequence in protein_seq_dicts.items():
    protein_class = PyProtein(sequence)
    CTD = protein_class.GetCTD()
    CTD = {'protein_name': protein_name, **CTD}
    ctd_features.append(CTD)

ctd_df = pd.DataFrame(ctd_features)
ctd_df = ctd_df[['protein_name'] + [col for col in ctd_df.columns if col != 'protein_name']]  # Ensure 'protein_name' is the first column
ctd_df.to_csv(f"{output_dir}/protein_descriptors.csv", index=False)
print(ctd_df.head())

# 4.전체 데이터에 대한 FingerPrint, Descriptor 계산.

In [ ]:
fpg = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

## molecule을 Morgan FingerPrint로 변환.
def compute_fingerprint(mol):
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    return BitVectToText(fp)  # Convert to BitString for storage


## molecule로부터 descriptor 계산.
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}
    descriptors = Descriptors.CalcMolDescriptors(mol)
    return descriptors


def process_row(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {'fingerprint': None, 'descriptors': {}}
    fingerprint = compute_fingerprint(mol)
    descriptors = calculate_descriptors(smiles)
    return {'fingerprint': fingerprint, 'descriptors': descriptors}

In [ ]:
OFFSET = 0
CHUNK_SIZE = 10000
train_parquet = f'{data_dir}/train.parquet'
con = duckdb.connect()

output_dir = f"{data_dir}/preprocessed"
output_file = f"{output_dir}/train.parquet"
os.makedirs(output_dir, exist_ok=True)

ctd_df = pd.read_csv(f"{data_dir}/protein_desc/protein_descriptors.csv")

num_workers = cpu_count()
pool = Pool(num_workers)

first_chunk = True
# while OFFSET < 20000:
while True:
    start_time = time.time()
    
    chunk = con.execute(f"""
    SELECT *
    FROM parquet_scan('{train_parquet}')
    LIMIT {CHUNK_SIZE} OFFSET {OFFSET}
    """).fetch_df()

    if chunk.empty:
        break

    smiles_list = chunk['molecule_smiles'].tolist()
    
    # 병렬로 데이터 처리
    results = pool.map(process_row, smiles_list)

    # 결과를 데이터프레임으로 변환
    fingerprints = [result['fingerprint'] for result in results]
    descriptors_list = [result['descriptors'] for result in results]
    
    chunk['fingerprints'] = fingerprints
    descriptor_df = pd.DataFrame(descriptors_list)
    excluded_descriptors = descriptor_df.columns[descriptor_df.isna().any()].tolist()
    descriptor_df.drop(columns=excluded_descriptors, inplace=True)
    used_descriptor = descriptor_df.columns.tolist()

    if first_chunk:
        print(f"제외된 descriptors: {excluded_descriptors}")
        print(f"사용된 descriptors: {used_descriptor}")

    # CTD 데이터 병합 (protein_name 기준)
    merged_chunk = pd.merge(chunk, ctd_df, on='protein_name', how='left')
    merged_chunk = pd.concat([merged_chunk, descriptor_df], axis=1)
    
    table = pa.Table.from_pandas(merged_chunk)

    if first_chunk:
        writer = pq.ParquetWriter(output_file, table.schema)
        first_chunk = False

    writer.write_table(table)
    OFFSET += CHUNK_SIZE

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Processed offset: {OFFSET} saved to {output_file}. Time taken: {elapsed_time:.2f} seconds")

pool.close()
pool.join()

writer.close()
con.close()

# 5.binds=1인 데이터의 총량으로 10 fold 구축하기

In [12]:
## molecule을 Morgan FingerPrint로 변환.
def compute_fingerprint(mol):
    if mol is None:
        return None
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
    return BitVectToText(fp)  # Convert to BitString for storage


## molecule로부터 descriptor 계산.
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {}
    descriptors = Descriptors.CalcMolDescriptors(mol)
    return descriptors


def process_row(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {'fingerprint': None, 'descriptors': {}}
    fingerprint = compute_fingerprint(mol)
    descriptors = calculate_descriptors(smiles)
    return {'fingerprint': fingerprint, 'descriptors': descriptors}

In [13]:
con = duckdb.connect()

## binds=0인 데이터만 load
count_binds_0 = con.query(f"""SELECT COUNT(*) 
                              FROM parquet_scan('{train_parquet}') 
                              WHERE binds = 0""").fetchone()[0]
print(f"Total binds=0 : {count_binds_0}")

## binds=1인 데이터만 load
count_binds_1 = con.query(f"""SELECT COUNT(*) 
                              FROM parquet_scan('{train_parquet}') 
                              WHERE binds = 1""").fetchone()[0]
print(f"Total binds=1 : {count_binds_1}")

## 전체 데이터 수
total_count = count_binds_0 + count_binds_1
print(f"Total data : {total_count}")

con.close()

Total binds=0 : 293656924
Total binds=1 : 1589906
Total data : 295246830


In [14]:
b0_chunk_size = count_binds_0 // 1800
b1_chunk_size = count_binds_1 // 10
print(b0_chunk_size)
print(b1_chunk_size)

163142
158990


In [15]:
output_dir = f"{data_dir}/preprocessed"
ctd_df = pd.read_csv(f"{data_dir}/protein_desc/protein_descriptors.csv")

In [19]:
def class_per_data_spliter(binds, chunk_size, parquet_path, ctd_df, process_row, output_path, max_chunk=0):
    offset = 0
    chunk_number = 0

    num_workers = cpu_count()
    pool = Pool(num_workers)

    fingerprints_folder = os.path.join(output_path, 'fingerprints')
    descriptors_folder = os.path.join(output_path, 'descriptors')
    os.makedirs(fingerprints_folder, exist_ok=True)
    os.makedirs(descriptors_folder, exist_ok=True)

    encoder = OneHotEncoder(sparse=False)
    encoder.fit(ctd_df[['protein_name']])  # ctd_df를 이용해 fit

    con = duckdb.connect()
    while True:
        if max_chunk and chunk_number == max_chunk:
            break

        start_time = time.time()

        # Phase 1: 데이터 읽기
        phase1_start = time.time()
        chunk = con.execute(f"""
        SELECT *
        FROM parquet_scan('{parquet_path}')
        WHERE binds = {binds}
        LIMIT {chunk_size}
        OFFSET {offset}
        """).fetch_df()
        phase1_end = time.time()
        print(f"Phase 1: Data read - Time taken: {phase1_end - phase1_start:.2f} seconds")

        if chunk.empty:
            break

        smiles_list = chunk['molecule_smiles'].tolist()
        protein_names = chunk['protein_name'].tolist()

        # Phase 2: 병렬 데이터 처리
        phase2_start = time.time()
        results = pool.map(process_row, smiles_list)
        phase2_end = time.time()
        print(f"Phase 2: Parallel processing - Time taken: {phase2_end - phase2_start:.2f} seconds")

        # Phase 3: 결과를 데이터프레임으로 변환
        phase3_start = time.time()
        fingerprints = [result['fingerprint'] for result in results]
        descriptors_list = [result['descriptors'] for result in results]
        phase3_end = time.time()
        print(f"Phase 3: Convert results to DataFrame - Time taken: {phase3_end - phase3_start:.2f} seconds")

        # Phase 4: fingerprints 저장
        phase4_start = time.time()
        fingerprints_df = pd.DataFrame(fingerprints)
        fingerprints_df['protein_name'] = protein_names
        fingerprints_df['binds'] = binds

        # CTD 데이터 병합 (fingerprints와 연결)
        merged_fingerprints = pd.merge(fingerprints_df, ctd_df, on='protein_name', how='left')

        # 원핫 인코딩 적용 (fingerprints)
        protein_name_encoded_fingerprints = encoder.transform(merged_fingerprints[['protein_name']])
        protein_name_encoded_df_fingerprints = pd.DataFrame(protein_name_encoded_fingerprints, columns=encoder.get_feature_names_out(['protein_name']))
        merged_fingerprints = pd.concat([merged_fingerprints, protein_name_encoded_df_fingerprints], axis=1).drop(columns=['protein_name'])

        # 제거할 컬럼이 존재하는지 확인 후 삭제
        columns_to_drop = ['buildingblock1_smiles', 'buildingblock2_smiles', 'buildingblock3_smiles', 'molecule_smiles']
        merged_fingerprints = merged_fingerprints.drop(columns=[col for col in columns_to_drop if col in merged_fingerprints.columns])

        fingerprints_file = os.path.join(fingerprints_folder, f'fingerprints_b{binds}_chunk_{chunk_number}.csv')
        merged_fingerprints.to_csv(fingerprints_file, index=False)
        phase4_end = time.time()
        print(f"Phase 4: Save fingerprints - Time taken: {phase4_end - phase4_start:.2f} seconds")
        print(fingerprints_file)

        # Phase 5: descriptors 데이터프레임 생성 및 protein_name 추가
        phase5_start = time.time()
        descriptor_df = pd.DataFrame(descriptors_list)
        descriptor_df['protein_name'] = protein_names
        descriptor_df['binds'] = binds
        excluded_descriptors = descriptor_df.columns[descriptor_df.isna().any()].tolist()
        descriptor_df.drop(columns=excluded_descriptors, inplace=True)
        used_descriptor = descriptor_df.columns.tolist()

        if chunk_number == 0:
            print(f"제외된 descriptors: {excluded_descriptors}")
            print(f"사용된 descriptors: {used_descriptor}")

        # CTD 데이터 병합 (descriptor와 연결)
        merged_descriptors = pd.merge(descriptor_df, ctd_df, on='protein_name', how='left')

        # 원핫 인코딩 적용 (descriptors)
        protein_name_encoded_descriptors = encoder.transform(merged_descriptors[['protein_name']])
        protein_name_encoded_df_descriptors = pd.DataFrame(protein_name_encoded_descriptors, columns=encoder.get_feature_names_out(['protein_name']))
        merged_descriptors = pd.concat([merged_descriptors, protein_name_encoded_df_descriptors], axis=1).drop(columns=['protein_name'])

        descriptors_file = os.path.join(descriptors_folder, f'descriptors_b{binds}_chunk_{chunk_number}.csv')
        merged_descriptors.to_csv(descriptors_file, index=False)
        phase5_end = time.time()
        print(f"Phase 5: Save descriptors - Time taken: {phase5_end - phase5_start:.2f} seconds")
        print(descriptors_file)

        offset += chunk_size
        chunk_number += 1

        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Processed chunk: {chunk_number}, offset: {offset}, Total Time taken: {elapsed_time:.2f} seconds \n")

    pool.close()
    pool.join()
    con.close()


In [20]:
class_per_data_spliter(binds=1, 
                       chunk_size=b1_chunk_size, 
                       parquet_path=train_parquet, 
                       ctd_df=ctd_df, 
                       process_row=process_row, 
                       output_path=f"{output_dir}/binds1",
                       max_chunk=10)

Phase 1: Data read - Time taken: 1.30 seconds
Phase 2: Parallel processing - Time taken: 77.29 seconds
Phase 3: Convert results to DataFrame - Time taken: 0.04 seconds
Phase 4: Save fingerprints - Time taken: 6.64 seconds
/home/pervinco/Datasets/leash-bio/preprocessed/binds1/fingerprints/fingerprints_b1_chunk_0.csv
제외된 descriptors: ['MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW']
사용된 descriptors: ['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 

In [ ]:
class_per_data_spliter(binds=0, 
                       chunk_size=b0_chunk_size, 
                       parquet_path=train_parquet, 
                       ctd_df=ctd_df, 
                       process_row=process_row, 
                       output_path=f"{output_dir}/binds0",
                       max_chunk=10)

Phase 1: Data read - Time taken: 0.09 seconds
Phase 2: Parallel processing - Time taken: 76.75 seconds
Phase 3: Convert results to DataFrame - Time taken: 0.04 seconds
Phase 4: Save fingerprints - Time taken: 6.75 seconds
제외된 descriptors: ['MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW']
사용된 descriptors: ['MaxAbsEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MinEStateIndex', 'qed', 'SPS', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'AvgIpc', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14',